In [1]:
import pandas as pd
import numpy as np
import tensorflow.keras as tf

In [2]:
# this file has a lot of sparse rows so we open it in this way
with open('/home/lisi/impact-prediction/data/paperHistories-1975-2005-2015-2.tsv', 'r') as f:
    lines = [row.split() for row in f.read().split("\n") if row]
# we find the longest list in lines
def find_max_list_idx(list):
    list_len = [len(i) for i in list]
    return np.argmax(np.array(list_len)) # retuns the index of the biggest list
# we need to make each list the same length so we pad with zeros the other entries
biggest_len = len(lines[find_max_list_idx(lines)])
paperHistories = {}
for i in range(len(lines)):
    paperHistories[i] = {'padding': lines[i] + ['0'] * (biggest_len - len(lines[i])), # add the zeroes at the end
                         'original': ['1'] * len(lines[i]) + ['0'] * (biggest_len - len(lines[i]))  # 1's for original values, 0's for the padded ones
                         }

In [3]:
df = pd.DataFrame(paperHistories).transpose()

In [4]:
more_than_14yeas = df.original.apply(lambda x: True if sum(list(map(int, x)))>=15 else False)

In [5]:
paperFeatures = pd.read_csv('/home/lisi/impact-prediction/data/paperFeatures-1975-2005-2015-2.tsv', sep='\t')
paperIds = pd.read_csv('/home/lisi/impact-prediction/data/paperIds-1975-2005-2015-2.tsv', sep='\t', names=['paperIds'])

In [6]:
df = df.join(paperIds)

In [7]:
# edgelist = pd.read_csv('/home/lisi/Desktop/Laci/impact_data/edgelist.csv')

In [8]:
# edgelist = edgelist.drop('target',axis=1).drop_duplicates().reset_index(drop=True)

In [9]:
# df = pd.merge(df, edgelist, left_on=['paperIds'], right_on=['source'], how='left').drop(['source'],axis=1)

In [10]:
# we read the citation graph and make a csv file
with open('/home/lisi/impact-prediction/data/paperIdToPaperData-1975-2015.json', 'r',encoding='utf-8') as j:
        d = j.read().replace('false', '0').replace('true', '1').splitlines()

In [11]:
# string to dictionary
good = []
bad = [] # there are some authors that have special characters that utf-8 or utf-16 cant solve
for i in d:
    try:
        good.append(eval(i))
    except:
        bad.append(i)

In [12]:
# manually added the bad rows
add_bad = {'id':['ca9ba78b3e268ec9b58ffa69524e6f35c0314fcc', 
                 '30af66277400354f88d0a5de0feaab9fb5278705', 
                '61afe58c68b91d20a65bb4166cfceb501745c134', 
                '6d46189d2040a01ccdb4236b0095361294b1a8bf'],
          'year':[2007, 2007, 2006, 2004],
          'venue':['','','','']}

In [13]:
metadata = pd.DataFrame.from_dict(good, orient='columns') # list to dataframe
metadata = metadata.drop(['citations-in-year', 'authors', 'is-survey'],axis=1)
metadata = metadata.append(pd.DataFrame(add_bad), ignore_index=True) # add the bad rows

In [14]:
df = pd.merge(df, metadata, left_on=['paperIds'], right_on=['id'], how='left').drop(['id'],axis=1)

In [15]:
df= df.iloc[more_than_14yeas.to_list(),:].reset_index(drop=True)

In [16]:
df['venue'].replace('', np.nan, inplace=True)
df.dropna(subset=['venue'], inplace=True)

In [17]:
df.padding = df.padding.apply(lambda x: list(map(int, x))[:15])

In [18]:
df.drop(['original'], axis=1, inplace=True)

In [19]:
years = df['padding'].apply(pd.Series)
years.columns = ['c{}'.format(i+1) for i in range(years.shape[1])]

In [20]:
df = df.join(years)

In [21]:
df.drop(['padding', 'paperIds'], axis=1, inplace=True)

In [22]:
df.to_csv('rnn_processed_data.csv', index=False)